### assignment_1 - task1

In [19]:
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import time

In [20]:
def dtw(s, t, window):
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)]) # warping cannot be less than the difference in lengths. 
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix[-1,-1] 

## 1-NN Classifer（DTW）
task1.1 Use the DTW function in the notebook to implement a simple 1-NN classifier for time-series data.
I defined this classifer_1NN function

In [21]:
#task 1.1 define an 1-NN classifer
def classifer_1NN(data_test, data_train, train_label, window):
    #Parameters： x_test, x_train, y_train, window
    dtw_dis = []
    for i in range(len(data_test)):
        label_dis = []
        min_dis = dtw(data_test[i], data_train[0], window)
        min_index = 0
        for j in range(len(data_train)):
            dis = dtw(data_test[i], data_train[j], window)
#             print("i,j = ",i,j,"dis:",dis)
            if(min_dis > dis):
                min_dis = dis
                min_index = j
        #dtw_dis.append([train_label[min_index],min_dis])
        dtw_dis.append(train_label[min_index])
    
#     print(dtw_dis,"\n",len(dtw_dis))
    return dtw_dis

### import data "UMD_TEST.txt"

In [22]:
#import data
umd = pd.read_csv("./UMD_TEST.txt", sep=r"\s+",header = None)
umd

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,1.0,0.017644,0.030949,0.050555,0.044484,0.053277,0.041576,0.030947,0.027086,0.013764,...,0.024575,0.033780,0.026589,0.013932,0.024928,0.022589,0.038248,0.049838,0.053419,0.040420
1,1.0,0.041296,0.003551,0.027470,0.013158,0.009571,0.008074,0.043743,0.040592,0.012190,...,0.060539,0.046991,0.023586,0.001562,-0.002196,0.036730,0.039027,0.007754,0.004697,0.031440
2,1.0,-0.000720,0.013283,0.029450,0.045201,0.006317,0.018805,0.028901,0.013832,0.015240,...,0.016442,0.039508,0.015171,0.034708,0.010835,0.002942,0.006924,0.029502,0.040786,0.023144
3,1.0,0.005201,0.013363,0.025733,0.026653,0.038946,0.012494,0.028303,0.032011,0.009467,...,0.006383,0.037448,0.044335,0.011143,-0.003624,0.001467,0.020991,0.027675,0.001621,0.015858
4,1.0,0.022926,0.027036,0.011668,0.019500,0.036049,-0.001297,0.019717,0.039583,0.020628,...,0.026997,0.036653,0.018117,0.018314,0.012536,0.040599,0.016590,0.032730,0.002498,0.011260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,3.0,0.008298,0.017240,0.010756,0.049488,0.005331,0.036829,0.045530,0.046181,0.044288,...,0.015062,0.028319,0.028443,0.051677,0.036134,0.038030,0.047388,0.006875,0.052691,0.051594
140,3.0,0.004801,0.042964,0.034519,-0.003426,-0.003353,0.005231,0.002309,0.029905,0.011177,...,0.017705,0.045264,0.041434,0.040670,0.034613,0.001551,0.017716,0.001042,0.047787,0.020318
141,3.0,0.033658,0.038235,0.009848,0.016061,0.026287,0.029979,0.024099,0.023647,0.017566,...,0.019306,0.022595,0.014942,0.009259,0.019186,0.035905,0.031095,0.006592,0.017948,-0.003614
142,3.0,0.022805,0.020270,0.015546,0.038309,0.016303,0.011456,0.018286,0.015730,0.007319,...,0.013016,0.030138,0.025736,0.005955,0.050157,0.014435,0.005638,0.036126,0.010751,-0.005682


In [23]:
#seperate feature and target
label = []
feature = []
for i in range(len(umd)):
    label.append(umd.values[i][0])  # true label
    feature.append(umd.values[i][1:])

label = np.array(label)
feature = np.array(feature)

print("label:\n",label)
print("feature:\n",feature)

label:
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
feature:
 [[ 0.01764446  0.03094927  0.05055511 ...  0.04983778  0.05341944
   0.04042037]
 [ 0.0412961   0.00355094  0.02747018 ...  0.00775355  0.00469676
   0.03144026]
 [-0.00071981  0.01328251  0.02945012 ...  0.02950245  0.0407861
   0.02314421]
 ...
 [ 0.03365773  0.03823499  0.00984808 ...  0.00659213  0.0179477
  -0.00361412]
 [ 0.02280525  0.02027041  0.01554644 ...  0.03612612  0.01075145
  -0.00568218]
 [ 0.0123563   0.02396996  0.0131491  ...  0.03790152  0.0086877
   0.0363948 ]]


In [24]:
#2) Dataset partitioning
    # parameters (feature value, target value); random_state is the random number seed
x_train, x_test, y_train, y_test = train_test_split(feature,label,test_size=0.33, random_state=42)
# x_train, x_test, y_train, y_test = train_test_split(feature,label)

print("y_test:\n", y_test)



y_test:
 [3. 1. 2. 3. 2. 1. 3. 2. 2. 1. 2. 2. 2. 3. 3. 2. 1. 2. 3. 2. 1. 3. 3. 3.
 2. 1. 1. 1. 1. 3. 1. 3. 1. 1. 1. 2. 3. 1. 3. 3. 3. 2. 1. 2. 1. 1. 2. 1.]


## Task 1-2 test performance
performance function can evaluate the model: Direct comparison of true and predicted values

In [25]:
def performance(y_test, estimator):
    # Model evaluation
    # Direct comparison of true and predicted values
    results = []
    count = 0
    for i in range(len(y_test)):
        if(y_test[i] == estimator[i]):
            results.append(True)
            count += 1
        else:
            results.append(False)
    score = count/len(y_test)
    print("result:\n",results)
    print("score:\n",count/len(y_test))
    return score

In [26]:
def knn_data(w):
    # KNN estimator
    estimator_dtw = classifer_1NN(x_test, x_train, y_train,window = w)
    print("estimator_dtw:\n",estimator_dtw)

    # Model evaluation
#     print("performance of DTW:\n")
    score_dtw = performance(y_test, estimator_dtw)
    
    return score_dtw

the code below can get forecast results and run times

result: 
when window = 4, test_size=0.33, random_state=42

score:
 0.9583333333333334
run_time:
 90.58415198326111  s

In [27]:
start_time = time.time()
score = knn_data(w = 4) #window = 4
end_time = time.time()
print("run_time:\n",end_time - start_time," s")

estimator_dtw:
 [3.0, 1.0, 2.0, 3.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 3.0, 3.0, 2.0, 1.0, 2.0, 3.0, 2.0, 1.0, 3.0, 3.0, 3.0, 2.0, 1.0, 1.0, 1.0, 2.0, 3.0, 1.0, 3.0, 1.0, 1.0, 1.0, 2.0, 3.0, 1.0, 3.0, 3.0, 3.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0]
result:
 [True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
score:
 0.9583333333333334
run_time:
 90.58415198326111  s


## task 1-3 1-NN Classifer (Euclidean)
Modify the function classifer_1NN I defined before, change the distance calculation function "DTW" used in the distance function to Euclidean

In [28]:
# create a new distance calculation function, use euclidean
def euclidean(test,train):
    dist = [(a - b)**2 for a, b in zip(test, train)]
    dist = math.sqrt(sum(dist))
    return dist

In [29]:
#task 1.3 1-NN classifer Euclidean
def classifer_1NN_Euclidean(data_test, data_train, train_label):
    dtw_dis = []
    for i in range(len(data_test)):
        label_dis = []
        min_dis = euclidean(data_test[i],data_train[0]) # use euclidean
        min_index = 0
        for j in range(len(data_train)):
            dis = euclidean(data_test[i],data_train[j])
#             print("i,j = ",i,j,"dis:",dis)
            if(min_dis > dis):
                min_dis = dis
                min_index = j
        #dtw_dis.append([train_label[min_index],min_dis])
        dtw_dis.append(train_label[min_index])
    
#     print(dtw_dis,"\n",len(dtw_dis))
    return dtw_dis

Euclidean performance
two methods: utilize scikit-learn or 1-NN euclidean classifier

In [30]:
#Euclidean performance
#Use the scikit-learn implementation for the Euclidean model

start_time = time.time()

# 4）KNN estimator
estimator = KNeighborsClassifier(n_neighbors = 1)
estimator.fit(x_train, y_train) #Put in the feature values of the training set and the target values for training

# calculate accuracy
score = estimator.score(x_test, y_test)
print("scikit-learn euclidean score:\n",score)

end_time = time.time()
print("run_time:\n",end_time - start_time," s")


scikit-learn euclidean score:
 0.8541666666666666
run_time:
 0.004984855651855469  s


In [31]:
#Euclidean performance
#use classifer_1NN_Euclidean
start_time = time.time()

estimator_euclidean = classifer_1NN_Euclidean(x_test, x_train, y_train)
print("estimator_euclidean:\n",estimator_euclidean)

#model evaluation
#Direct comparison of true and predicted values
print("performance of Euclidean:\n")
score_e = performance(y_test, estimator_euclidean)
print("score_e = ", score_e)

end_time = time.time()
print("run_time:\n",end_time - start_time," s")

estimator_euclidean:
 [3.0, 1.0, 2.0, 3.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 3.0, 3.0, 2.0, 1.0, 2.0, 3.0, 2.0, 1.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 1.0, 3.0, 1.0, 1.0, 1.0, 2.0, 3.0, 1.0, 3.0, 3.0, 3.0, 2.0, 3.0, 2.0, 1.0, 2.0, 2.0, 1.0]
performance of Euclidean:

result:
 [True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, False, True, True]
score:
 0.8541666666666666
score_e =  0.8541666666666666
run_time:
 0.47246623039245605  s


## result of comparing DTW with Euclidean
dtw:
score of dtw: 0.9583333333333334
run_time is 90.58415198326111 s
euclidean:
score: 0.8541666666666666
run_time:
 0.004984855651855469  s
run_time:
 0.47246623039245605 s
 
conclusion: the accuracy of dtw is better than that of euclidean, but efficiency of euclidean is much better than that of dtw

## best value for the window
At first I tested all the values of window from 0 to len(x_train), but found this to be unnecessary. I found that the score stopped changing once the window grew to a certain level (window > 69), and that the best score still occurred within window = 14

result:
best_window =  5 best_score =  0.9791666666666666

In [32]:
best_window = 0
best_score = 0
for window in range(15):
    score = knn_data(window)
    if(score>best_score):
        best_window = window
        best_score = score
    print("window = ", window, ", score = ",score)
print("best_window = ",best_window,"best_score = ",best_score)

estimator_dtw:
 [3.0, 1.0, 2.0, 3.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 2.0, 1.0, 2.0, 3.0, 2.0, 1.0, 3.0, 3.0, 3.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 3.0, 1.0, 1.0, 1.0, 2.0, 3.0, 1.0, 3.0, 3.0, 3.0, 2.0, 3.0, 2.0, 1.0, 2.0, 2.0, 1.0]
result:
 [True, True, True, True, False, True, False, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, False, True, True]
score:
 0.8541666666666666
window =  0 , score =  0.8541666666666666
estimator_dtw:
 [3.0, 1.0, 2.0, 3.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 2.0, 1.0, 2.0, 3.0, 2.0, 1.0, 3.0, 3.0, 3.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 3.0, 1.0, 1.0, 1.0, 2.0, 3.0, 1.0, 3.0, 3.0, 3.0, 2.0, 3.0, 2.0, 1.0, 2.0, 2.0, 1.0]
result:
 [True, True, True, True, False, True, False, True, True, True, True, True, True, False, True, True, Tru

estimator_dtw:
 [3.0, 1.0, 2.0, 3.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 3.0, 3.0, 2.0, 1.0, 2.0, 3.0, 2.0, 1.0, 3.0, 3.0, 3.0, 2.0, 1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 3.0, 1.0, 1.0, 1.0, 2.0, 3.0, 1.0, 3.0, 3.0, 3.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0]
result:
 [True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
score:
 0.9791666666666666
window =  14 , score =  0.9791666666666666
best_window =  5 best_score =  0.9791666666666666
